In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from pathlib import Path
from pynotate import Project
import numpy as np
from fundus_lesions_toolkit.models.segmentation import segment as segment_lesions, Dataset
from fundus_data_toolkit.functional import open_image

from fundus_lesions_toolkit.constants import LESIONS
from fundus_odmac_toolkit.models.segmentation import segment as segment_odmac
from tqdm.notebook import tqdm

segmentation_classes = ['Lesions/' + l for l in LESIONS[1:]] + ['OD', 'MAC']

classifications_classes = [{'name': 'Diabetic Retinopathy', 'classes': ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative']}]
classification_multilabel = {'name': 'Others diseases', 'classes': ['Hypertension', 'Glaucoma', 'Myopia', 'Other']}

def run_model(filepath):
    img = open_image(filepath)
    lesions = segment_lesions(img, train_datasets=Dataset.IDRID).argmax(0).cpu().numpy()
    od_mask = segment_odmac(img).argmax(0).cpu().numpy()
    # Lesions
    masks = [255*(lesions == i).astype(np.uint8) for i in range(1, 5)]
    # OD and MAC
    masks += [255*(od_mask==i) for i in range(1, 3)] 

    # Random classification
    multilabel = np.random.choice(classification_multilabel['classes'], size=np.random.randint(0, len(classification_multilabel['classes']))).tolist()
    multiclass = np.random.choice(classifications_classes[0]['classes'], size=1).tolist()
    multilabel = None if len(multilabel) == 0 else multilabel
    
    return masks, multiclass, multilabel


with Project(project_name="FundusLesions", 
             input_dir=str(Path("inputFundus/").resolve()),
             output_dir=str(Path(".").resolve()),
             classification_classes=classifications_classes,
             classification_multilabel=classification_multilabel,
             segmentation_classes=segmentation_classes) as cli:
    for i in tqdm(range(15)):
        filepath = f'C:/Users/cleme/Documents/demo/fundus/macula_{i}.jpeg'
        masks, multiclass, multilabel = run_model(filepath)
        cli.load_image(filepath, segmentation_masks=masks, multiclass_choices=multiclass, multilabel_choices=multilabel)



  0%|          | 0/15 [00:00<?, ?it/s]